In [ ]:
1. Общий долг на дату
-- 1.1 Общий долг на дату

SELECT business_dt, round(SUM(portf_total_amt_rub), 2) FROM prod_v_emart.invest_account_d
WHERE portf_total_amt_rub < 0 AND account_status_cd = 'ACT'
GROUP BY business_dt
ORDER BY business_dt DESC
LIMIT 30

In [ ]:
1.2 Изменение величины общего долга по БС за неделю

--1.2 Изменение величины общего долга по БС за неделю
-- Прим. Так как даты отсортированы в обратном порядке, используем агрегирующую ф-цию lead(), а не lag().
-- Нужно оптимизировать оконную ф-цию. Пока запрос с краткой записью падает с ошибкой.

SELECT 
    'Изменение с ' || iad.business_dt || ' по ' || (iad.business_dt - integer '7') AS debt_weekly_change, 
    iad.business_dt,
    iad.debt_on_dt,
    lead(iad.debt_on_dt, 7) OVER (PARTITION BY iad.partition_flg ORDER BY business_dt DESC) AS debt_week_ago,

    (iad.debt_on_dt - (lead(iad.debt_on_dt, 7) OVER (PARTITION BY iad.partition_flg ORDER BY business_dt DESC))) AS debt_weekly_change_amt,

    CASE 
         WHEN (iad.debt_on_dt - (lead(iad.debt_on_dt, 7) OVER (PARTITION BY iad.partition_flg ORDER BY business_dt DESC))) > 0 
            THEN ('Долг уменьшился на ' || (abs(iad.debt_on_dt - (lead(iad.debt_on_dt, 7) OVER (PARTITION BY iad.partition_flg ORDER BY business_dt DESC)))) || ' руб.')
         WHEN (iad.debt_on_dt - (lead(iad.debt_on_dt, 7) OVER (PARTITION BY iad.partition_flg ORDER BY business_dt DESC))) < 0 
            THEN ('Долг увеличился на ' || (abs(iad.debt_on_dt - (lead(iad.debt_on_dt, 7) OVER (PARTITION BY iad.partition_flg ORDER BY business_dt DESC)))) || ' руб.')
         ELSE 'Величина долга не изменилась'
    END AS debt_weekly_change_text
FROM (SELECT business_dt, round(SUM(portf_total_amt_rub), 2) AS debt_on_dt, 1 AS partition_flg
    FROM prod_v_emart.invest_account_d
    WHERE portf_total_amt_rub < 0 AND account_status_cd = 'ACT'
    GROUP BY business_dt
    ORDER BY business_dt DESC
    LIMIT 30) iad

In [ ]:
Данные для графика (величина общего долга и суммарный портфель всех клиентов   (за весь период (с 01.01.2021)).

--График, совмещающий две линии: суммарный портфель всех клиентов и величина общего долга (за весь период (с 01.01.2021)).
-- Вопрос: как считать (здесь и далее) суммарный портфель клиентов? Сейчас в сумме идут и отрицательные. Оставить как есть или учитывать только положительные?

SELECT iad1.business_dt, 
        abs(iad2.negative_portf_amt_incl_negative) AS total_debt, 
        round(SUM(portf_total_amt_rub), 2) AS all_clients_portf_amt_incl_negative, 
        iad3.all_clients_portf_amt_only_positive AS all_clients_portf_amt_only_positive,
        round((abs(iad2.negative_portf_amt_incl_negative) / round(SUM(portf_total_amt_rub), 2)) * 100, 6) AS total_debt_to_all_clients_incl_negative,
        round((abs(iad2.negative_portf_amt_incl_negative) / iad3.all_clients_portf_amt_only_positive) * 100, 6) AS total_debt_to_all_clients_only_positive

FROM prod_v_emart.invest_account_d iad1
INNER JOIN (SELECT business_dt, round(SUM(portf_total_amt_rub), 2) AS negative_portf_amt_incl_negative 
            FROM prod_v_emart.invest_account_d
            WHERE portf_total_amt_rub < 0 AND account_status_cd = 'ACT' AND business_dt > '2021-01-01'
            GROUP BY business_dt
            ORDER BY business_dt DESC) iad2 ON iad2.business_dt = iad1.business_dt
INNER JOIN (SELECT business_dt, round(SUM(portf_total_amt_rub), 2) AS all_clients_portf_amt_only_positive 
            FROM prod_v_emart.invest_account_d
            WHERE portf_total_amt_rub > 0 AND account_status_cd = 'ACT' AND business_dt > '2021-01-01'
            GROUP BY business_dt
            ORDER BY business_dt DESC) iad3 ON iad3.business_dt = iad1.business_dt
WHERE account_status_cd = 'ACT' AND iad1.business_dt > '2021-01-01'
GROUP BY iad1.business_dt, iad2.business_dt, iad2.negative_portf_amt_incl_negative, iad3.all_clients_portf_amt_only_positive
ORDER BY iad2.business_dt DESC

In [ ]:
2. Доля БС (от общего кол-ва БС с долгом) с долгом менее 10 тыс.
-- 2. Доля БС (от общего кол-ва БС с долгом) с долгом менее 10 тыс.: 

SELECT iad.business_dt, portf_debt_less_10000_num, count(portf_total_amt_rub) AS negative_portf_num, 
    round((CAST(portf_debt_less_10000_num AS numeric) / CAST(count(portf_total_amt_rub) AS numeric)) * 100, 2) AS portf_debt_less_10000_share
FROM prod_v_emart.invest_account_d iad
INNER JOIN    
    (SELECT business_dt, count(portf_total_amt_rub) AS portf_debt_less_10000_num
        FROM prod_v_emart.invest_account_d
        WHERE portf_total_amt_rub < 0 AND portf_total_amt_rub >= -10000 AND account_status_cd = 'ACT'
        GROUP BY business_dt
        ORDER BY business_dt DESC
        ) count_debt_less_10000
    ON iad.business_dt = count_debt_less_10000.business_dt
WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT'
GROUP BY iad.business_dt, portf_debt_less_10000_num
ORDER BY iad.business_dt DESC

In [ ]:
3. Доля БС (от общего кол-ва БС с долгом) с долгом более 10 тыс.
-- 3. Доля БС (от общего кол-ва БС с долгом) с долгом более 10 тыс.: 

SELECT iad.business_dt, portf_debt_more_10000_num, count(portf_total_amt_rub) AS negative_portf_num,  
    round((CAST(portf_debt_more_10000_num AS numeric) / CAST(count(portf_total_amt_rub) AS numeric)) * 100, 2) AS portf_debt_more_10000_share
FROM prod_v_emart.invest_account_d iad
INNER JOIN    
    (SELECT business_dt, count(portf_total_amt_rub) AS portf_debt_more_10000_num
        FROM prod_v_emart.invest_account_d
        WHERE portf_total_amt_rub < -10000 AND account_status_cd = 'ACT'
        GROUP BY business_dt
        ORDER BY business_dt DESC
        ) count_debt_more_10000
    ON iad.business_dt = count_debt_more_10000.business_dt
WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT'
GROUP BY iad.business_dt, portf_debt_more_10000_num
ORDER BY iad.business_dt DESC

In [ ]:
Новый recovery_rate_amt (по всем должникам)

WITH q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt >= '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' --сегодняшняя дата
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT round(SUM(q4.max_negative_portf_sum), 2) AS total_hist_debt, 
       (SELECT SUM(q5.negative_portfs_sum) FROM q5) AS debt_today,
        round( ((SUM(q4.max_negative_portf_sum) - (SELECT SUM(q5.negative_portfs_sum) FROM q5))  / SUM(q4.max_negative_portf_sum)) * 100, 2) AS recovery_rate_amt
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
    INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < 0
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
Новый recovery_rate_amt (долг до 10000 руб.)

WITH q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt >= '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' --сегодняшняя дата
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        HAVING SUM(iad.portf_total_amt_rub) < 0 AND SUM(iad.portf_total_amt_rub) >= -10000
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT SUM(q4.max_negative_portf_sum) AS total_debt_in_category, 
       (SELECT SUM(q5.negative_portfs_sum) FROM q5) AS debt_in_category_today,
        round( ((SUM(q4.max_negative_portf_sum) - (SELECT SUM(q5.negative_portfs_sum) FROM q5))  / SUM(q4.max_negative_portf_sum)) * 100, 2) AS recovery_rate_amt_10000 
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
    INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < 0 AND q2.max_negative_portf_sum >= -10000
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
Новый recovery_rate_amt (10-100 тыс. руб.)

WITH q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt > '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' --сегодняшняя дата
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        HAVING SUM(iad.portf_total_amt_rub) < -10000 AND SUM(iad.portf_total_amt_rub) >= -100000
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT SUM(q4.max_negative_portf_sum) AS total_debt_in_category, 
       (SELECT SUM(q5.negative_portfs_sum) FROM q5) AS debt_in_category_today,
        round( ((SUM(q4.max_negative_portf_sum) - (SELECT SUM(q5.negative_portfs_sum) FROM q5))  / SUM(q4.max_negative_portf_sum)) * 100, 2) AS recovery_rate_amt_100000 
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
    INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < -10000 AND q2.max_negative_portf_sum >= -100000
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
Новый recovery_rate_amt (100 тыс.- 1 млн руб.)

WITH q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt > '2021-01-01' iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' --сегодняшняя дата
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        HAVING SUM(iad.portf_total_amt_rub) < -100000 AND SUM(iad.portf_total_amt_rub) >= -1000000
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT SUM(q4.max_negative_portf_sum) AS total_debt_in_category, 
       (SELECT SUM(q5.negative_portfs_sum) FROM q5) AS debt_in_category_today,
        round( ((SUM(q4.max_negative_portf_sum) - (SELECT SUM(q5.negative_portfs_sum) FROM q5))  / SUM(q4.max_negative_portf_sum)) * 100, 2) AS recovery_rate_amt_1000000
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
    INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < -100000 AND q2.max_negative_portf_sum >= -1000000
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
Новый recovery_rate_amt (1+ млн руб.)

WITH q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt > '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' --сегодняшняя дата
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        HAVING SUM(iad.portf_total_amt_rub) < -1000000
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT SUM(q4.max_negative_portf_sum) AS total_debt_in_category, 
       (SELECT SUM(q5.negative_portfs_sum) FROM q5) AS debt_in_category_today,
        round( ((SUM(q4.max_negative_portf_sum) - (SELECT SUM(q5.negative_portfs_sum) FROM q5))  / SUM(q4.max_negative_portf_sum)) * 100, 2) AS recovery_rate_amt_mlnrs
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
    INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < -1000000
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
Новый recovery_rate_amt для клиентов с подключенным флагом маржинальной торговли (на день образования долга или за день)

WITH q_margin (party_rk) AS (SELECT party_rk 
                            FROM   (SELECT account_rk,  business_dt, party_rk, portf_total_amt_rub, lag(portf_total_amt_rub, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC) AS portf_total_amt_rub_yesterday,
                                        CASE 
                                            WHEN portf_total_amt_rub < 0 AND lag(portf_total_amt_rub, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC) >= 0 THEN 1
                                            ELSE 0
                                        END AS became_debtor_flg,
                                        margin_status_flg, 
                                        COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) AS margin_status_flg_yesterday,
                                        CASE 
                                            WHEN margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) = 2 THEN 1
                                            ELSE margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0)
                                        END AS is_margin
                                FROM prod_v_emart.invest_account_d iad
                                WHERE business_dt >= '2021-01-01' AND iad.account_status_cd = 'ACT'
                                ORDER BY account_rk, business_dt) iad1
                            WHERE became_debtor_flg = 1 AND is_margin = 1),

     q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt >= '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' 
                                                            AND iad.party_rk IN (SELECT party_rk FROM q_margin)
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT --round(SUM(q4.max_negative_portf_sum), 2) AS total_hist_debt, 
       --(SELECT SUM(q5.negative_portfs_sum) FROM q5) AS debt_today,
        round( ((SUM(q4.max_negative_portf_sum) - (SELECT SUM(q5.negative_portfs_sum) FROM q5))  / SUM(q4.max_negative_portf_sum)) * 100, 2) AS recovery_rate_amt_margin
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
    INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < 0 
        AND q2.party_rk IN (SELECT party_rk FROM q_margin) 
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
Новый recovery_rate_amt для клиентов БЕЗ маржинальной торговли (на день образования долга или за день)

WITH q_non_margin (party_rk) AS (SELECT party_rk 
                            FROM   (SELECT account_rk,  business_dt, party_rk,
                                        CASE 
                                            WHEN portf_total_amt_rub < 0 AND lag(portf_total_amt_rub, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC) >= 0 THEN 1
                                            ELSE 0
                                        END AS became_debtor_flg,
                                        CASE 
                                            WHEN margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) = 2 THEN 1
                                            ELSE margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0)
                                        END AS is_margin
                                FROM prod_v_emart.invest_account_d iad
                                WHERE business_dt >= '2021-01-01' AND iad.account_status_cd = 'ACT'
                                ORDER BY account_rk, business_dt) iad1
                            WHERE became_debtor_flg = 1 AND is_margin = 0),

     q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt >= '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' 
                                                            AND iad.party_rk IN (SELECT party_rk FROM q_non_margin)
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT --round(SUM(q4.max_negative_portf_sum), 2) AS total_hist_debt, 
       --(SELECT SUM(q5.negative_portfs_sum) FROM q5) AS debt_today,
        round( ((SUM(q4.max_negative_portf_sum) - (SELECT SUM(q5.negative_portfs_sum) FROM q5))  / SUM(q4.max_negative_portf_sum)) * 100, 2) AS recovery_rate_amt
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
    INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < 0 
        AND q2.party_rk IN (SELECT party_rk FROM q_non_margin) 
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
recovery_rate_num (по всем клиентам с задолженностью)
WITH q (today_debtors_num) AS (SELECT count(DISTINCT party_rk) AS today_debtors_num 
                                FROM prod_v_emart.invest_account_d iad1 
                                WHERE iad1.portf_total_amt_rub < 0 AND iad1.account_status_cd = 'ACT' AND iad1.business_dt = now()::date - integer '1') -- указать сегодняшнюю дату

SELECT count(DISTINCT party_rk) AS all_hist_debtors_num,
     (SELECT today_debtors_num FROM q) AS today_debtors_num,
     round((CAST((count(DISTINCT party_rk) - (SELECT today_debtors_num FROM q)) AS numeric) / CAST(count(DISTINCT party_rk) AS numeric)) * 100, 2) AS recovery_rate_num
FROM prod_v_emart.invest_account_d iad
WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt >= '2021-01-01' AND iad.business_dt <= now()::date - integer '1'


In [ ]:
recovery_rate_num_margin (по клиентам с подключенной марж.торговлей, по наличию флага марж.торг. 
                          в день образования долга и за день до)

WITH q_margin (party_rk) AS (SELECT party_rk 
                            FROM   (SELECT account_rk,  business_dt, party_rk,
                                        CASE 
                                            WHEN portf_total_amt_rub < 0 AND lag(portf_total_amt_rub, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC) >= 0 THEN 1
                                            ELSE 0
                                        END AS became_debtor_flg,
                                        CASE 
                                            WHEN margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) = 2 THEN 1
                                            ELSE margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0)
                                        END AS is_margin
                                FROM prod_v_emart.invest_account_d iad
                                WHERE business_dt >= '2021-01-01' AND iad.account_status_cd = 'ACT'
                                ORDER BY account_rk, business_dt) iad1
                            WHERE became_debtor_flg = 1 AND is_margin = 1),

     q (today_debtors_num) AS (SELECT count(DISTINCT party_rk) AS today_debtors_num 
                                FROM prod_v_emart.invest_account_d iad1 
                                WHERE iad1.portf_total_amt_rub < 0 AND iad1.account_status_cd = 'ACT' AND iad1.business_dt = now()::date - integer '1' 
                                        AND iad1.party_rk IN (SELECT party_rk FROM q_margin))
                                    
SELECT --count(DISTINCT party_rk) AS all_hist_debtors_num,
     --(SELECT today_debtors_num FROM q) AS today_debtors_num,
     round((CAST((count(DISTINCT party_rk) - (SELECT today_debtors_num FROM q)) AS numeric) / CAST(count(DISTINCT party_rk) AS numeric)) * 100, 2) AS recovery_rate_num
FROM prod_v_emart.invest_account_d iad
WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt >= '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
        AND party_rk IN (SELECT party_rk FROM q_margin)

In [ ]:
recovery_rate_num_non_margin (по клиентам без подключенной марж.торговли)

WITH q_non_margin (party_rk) as (SELECT party_rk 
                            FROM   (SELECT account_rk,  business_dt, party_rk,
                                        CASE 
                                            WHEN portf_total_amt_rub < 0 AND lag(portf_total_amt_rub, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC) >= 0 THEN 1
                                            ELSE 0
                                        END AS became_debtor_flg,
                                        CASE 
                                            WHEN margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) = 2 THEN 1
                                            ELSE margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0)
                                        END AS is_margin
                                FROM prod_v_emart.invest_account_d iad
                                WHERE business_dt >= '2021-01-01' AND iad.account_status_cd = 'ACT'
                                ORDER BY account_rk, business_dt) iad1
                            WHERE became_debtor_flg = 1 AND is_margin = 0),

     q (today_debtors_num) AS (SELECT count(DISTINCT party_rk) AS today_debtors_num 
                                FROM prod_v_emart.invest_account_d iad1 
                                WHERE iad1.portf_total_amt_rub < 0 AND iad1.account_status_cd = 'ACT' AND iad1.business_dt = now()::date - integer '1' -- указать сегодняшнюю дату
                                    AND party_rk IN (SELECT party_rk FROM q_non_margin))
                                    
SELECT --count(DISTINCT party_rk) AS all_hist_debtors_num,
     --(SELECT today_debtors_num FROM q) AS today_debtors_num,
     round((CAST((count(DISTINCT party_rk) - (SELECT today_debtors_num FROM q)) AS numeric) / CAST(count(DISTINCT party_rk) AS numeric)) * 100, 2) AS recovery_rate_num
FROM prod_v_emart.invest_account_d iad
WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt >= '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
        AND party_rk IN (SELECT party_rk FROM q_non_margin)

In [ ]:
recovery_rate_num (клиенты с долгом до 10 тыс.руб.)

WITH q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt > '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' --сегодняшняя дата
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        HAVING SUM(iad.portf_total_amt_rub) < 0 AND SUM(iad.portf_total_amt_rub) >= -10000
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT count(q4.party_rk) AS hist_all_debtors_in_category_num, 
       (SELECT count(party_rk) FROM q5) AS debtors_in_category_today_num,
        round((CAST(count(q4.party_rk) - (SELECT count(party_rk) FROM q5) AS numeric) / CAST(count(q4.party_rk) AS numeric) * 100), 2) AS recovery_rate_num_10000 
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
    INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < 0 AND q2.max_negative_portf_sum >= -10000
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
recovery_rate_num (клиенты с долгом 10-100 тыс.руб.)

WITH q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt > '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' --сегодняшняя дата
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        HAVING SUM(iad.portf_total_amt_rub) < -10000 AND SUM(iad.portf_total_amt_rub) >= -100000
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT count(q4.party_rk) AS hist_all_debtors_in_category_num, 
       (SELECT count(party_rk) FROM q5) AS debtors_in_category_today_num,
        round((CAST(count(q4.party_rk) - (SELECT count(party_rk) FROM q5) AS numeric) / CAST(count(q4.party_rk) AS numeric) * 100), 2) AS recovery_rate_num_100000 
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
        INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < -10000 AND q2.max_negative_portf_sum >= -100000
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
recovery_rate_num (клиенты с долгом 100 тыс.-1 млн руб.)

WITH q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt > '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1' --сегодняшняя дата
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        HAVING SUM(iad.portf_total_amt_rub) < -100000 AND SUM(iad.portf_total_amt_rub) >= -1000000
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT count(q4.party_rk) AS hist_all_debtors_in_category_num, 
       (SELECT count(party_rk) FROM q5) AS debtors_in_category_today_num,
        round((CAST(count(q4.party_rk) - (SELECT count(party_rk) FROM q5) AS numeric) / CAST(count(q4.party_rk) AS numeric) * 100), 2) AS recovery_rate_num_1000000 
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
        INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < -100000 AND q2.max_negative_portf_sum >= -1000000
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
recovery_rate_num (клиенты с долгом 1+ млн руб.)

WITH q (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt > '2021-01-01' AND iad.business_dt <= now()::date - integer '1'
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        ORDER BY iad.party_rk, iad.business_dt),
     q2 (party_rk, max_negative_portf_sum) AS (SELECT q.party_rk, MIN(q.negative_portfs_sum) AS max_negative_portf_sum FROM q GROUP BY q.party_rk),
     q5 (party_rk, business_dt, negative_portfs_sum) AS (SELECT iad.party_rk, iad.business_dt, SUM(iad.portf_total_amt_rub) AS negative_portfs_sum 
                                                        FROM prod_v_emart.invest_account_d iad
                                                        WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1'  --сегодняшняя дата
                                                        GROUP BY iad.party_rk, iad.business_dt
                                                        HAVING SUM(iad.portf_total_amt_rub) < -1000000
                                                        ORDER BY iad.party_rk, iad.business_dt)

SELECT count(q4.party_rk) AS hist_all_debtors_in_category_num, 
       (SELECT count(party_rk) FROM q5) AS debtors_in_category_today_num,
        round((CAST(count(q4.party_rk) - (SELECT count(party_rk) FROM q5) AS numeric) / CAST(count(q4.party_rk) AS numeric) * 100), 2) AS recovery_rate_num_mlnrs 
FROM (SELECT q2.party_rk, MIN(q3.business_dt) AS max_debt_date, q2.max_negative_portf_sum FROM q2
        INNER JOIN (SELECT * FROM q) q3 ON q3.party_rk = q2.party_rk AND q2.max_negative_portf_sum = q3.negative_portfs_sum
    WHERE q2.max_negative_portf_sum < -1000000
    GROUP BY q2.party_rk, q2.max_negative_portf_sum) q4

In [ ]:
--5.1. Процент клиентов с долгом от общего кол-ва активных клиентов на сегодня
--5.1. Процент клиентов с долгом от общего кол-ва активных клиентов на сегодня:
    
SELECT iad.business_dt, count(DISTINCT party_rk) AS negative_clients_num, iad2.all_clients_num, 
        (round((CAST(count(DISTINCT party_rk) AS numeric) / CAST(iad2.all_clients_num AS numeric)), 8) * 100) AS debt_clients_to_all_share 
FROM prod_v_emart.invest_account_d iad
INNER JOIN 
    (SELECT business_dt, count(DISTINCT party_rk) AS all_clients_num FROM prod_v_emart.invest_account_d 
    WHERE account_status_cd = 'ACT'
    GROUP BY business_dt
    ORDER BY business_dt DESC
    LIMIT 30) iad2
ON iad2.business_dt = iad.business_dt
WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT'
GROUP BY iad.business_dt, iad2.all_clients_num
ORDER BY iad.business_dt DESC
LIMIT 30

In [ ]:
5.2 Отношение общего долга к сумме всех активных портфелей
--5.2 Отношение общего долга к сумме всех активных портфелей:
    
SELECT iad.business_dt, -sum(iad.portf_total_amt_rub) AS negative_portf_amt, iad2.all_clients_portf_amt, 
        (round((CAST(-sum(iad.portf_total_amt_rub) AS numeric) / CAST(iad2.all_clients_portf_amt AS numeric)), 8) * 100) AS debt_to_all_portf_amt_share 
FROM prod_v_emart.invest_account_d iad
INNER JOIN 
    (SELECT business_dt, sum(portf_total_amt_rub) AS all_clients_portf_amt FROM prod_v_emart.invest_account_d 
    WHERE account_status_cd = 'ACT'
    GROUP BY business_dt
    ORDER BY business_dt DESC
    LIMIT 30) iad2
ON iad2.business_dt = iad.business_dt
WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT'
GROUP BY iad.business_dt, iad2.all_clients_portf_amt
ORDER BY iad.business_dt DESC
LIMIT 30

In [ ]:
6.1. Отношение суммы долга к количеству активных БС:
--6.1. Отношение суммы долга к количеству активных БС

SELECT iad.business_dt, -sum(iad.portf_total_amt_rub) AS negative_portf_amt, iad2.all_accs_num, 
        (round((CAST(-sum(iad.portf_total_amt_rub) AS numeric) / CAST(iad2.all_accs_num AS numeric)), 2)) AS debt_per_client 
FROM prod_v_emart.invest_account_d iad
INNER JOIN 
    (SELECT business_dt, count(portf_total_amt_rub) AS all_accs_num FROM prod_v_emart.invest_account_d 
    WHERE account_status_cd = 'ACT'
    GROUP BY business_dt
    ORDER BY business_dt DESC
    LIMIT 30) iad2
ON iad2.business_dt = iad.business_dt
WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT'
GROUP BY iad.business_dt, iad2.all_accs_num
ORDER BY iad.business_dt DESC
LIMIT 30

In [ ]:
--6.2. Отношение суммы долга к кол-ву счетов с подключенной марж.торговлей. 
--6.2. Отношение суммы долга к кол-ву счетов с подключенной марж.торговлей. 
-- Экон.смысл: какой риск даёт каждый новый счёт с вкл.маржиналкой. Как "размываются" риски по счетам с марж.торговлей.
--Вопрос: А нужно ли в этом соотношении отслеживать вчерашний флаг марж.торговли? В recovery_rate это обосновано, т.к. важен момент образования долга (и возможны манипуляции). Здесь же просто показатель на текущий день. --решили, что не надо (в p.s. написано почему)
--P.S. В конце посчитано количество клиентов с маржиналкой на каждый день с учётом флагов за сегодня и вчера - результат отличается незначительно (в контексте метрики 6.2 уж точно).

SELECT iad.business_dt, -sum(iad.portf_total_amt_rub) AS negative_portf_amt, 
       iad2.margin_accs_num, 
       round((CAST(-sum(iad.portf_total_amt_rub) AS numeric) / CAST(iad2.margin_accs_num AS numeric)), 2) AS debt_per_margin_acc

FROM prod_v_emart.invest_account_d iad
INNER JOIN 
    (SELECT business_dt, count(account_rk) AS margin_accs_num FROM prod_v_emart.invest_account_d 
    WHERE account_status_cd = 'ACT' AND margin_status_flg = 1
    GROUP BY business_dt
    ORDER BY business_dt DESC
    LIMIT 30) iad2
ON iad2.business_dt = iad.business_dt
WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT'
GROUP BY iad.business_dt, iad2.margin_accs_num
ORDER BY iad.business_dt DESC
LIMIT 30

In [ ]:
Количество долга, приходящегося на один маржинальный БС с долгом 
(взяты все БС с марж.торговлей на день/за день до формирования долга)

SELECT iad.business_dt, -sum(iad.portf_total_amt_rub) AS negative_portf_amt, 
       count(account_rk), 
       round((CAST(-sum(iad.portf_total_amt_rub) AS numeric) / CAST(count(account_rk) AS numeric)), 2) AS debt_per_margin_acc

FROM prod_v_emart.invest_account_d iad

WHERE iad.portf_total_amt_rub < 0 AND iad.account_status_cd = 'ACT' AND iad.business_dt = now()::date - integer '1'
        AND account_rk IN (SELECT account_rk 
                            FROM    (SELECT account_rk,  business_dt, party_rk,
                                                CASE 
                                                    WHEN portf_total_amt_rub < 0 AND lag(portf_total_amt_rub, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC) >= 0 THEN 1
                                                    ELSE 0
                                                END AS became_debtor_flg,
                                                CASE 
                                                    WHEN margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) = 2 THEN 1
                                                    ELSE margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0)
                                                END AS is_margin
                                        FROM prod_v_emart.invest_account_d iad
                                        WHERE business_dt >= '2021-01-01' AND iad.account_status_cd = 'ACT' -- AND account_rk = '42473233'
                                        ORDER BY account_rk, business_dt) iad1
                                    WHERE became_debtor_flg = 1 AND is_margin = 1)
GROUP BY iad.business_dt
ORDER BY iad.business_dt DESC

In [ ]:
--отслеживание статуса маржинальной торговли (за сегодня и вчера)
--отслеживаем у БС (+ подтягиваем party_rk клиентов) статус маржинальной торговли (если или вчера, или сегодня была включена марж.торговля - БС маржинальный), 
--смотрим на статус в день, когда образовалась задолженность (became_debtor_flg)

SELECT account_rk, business_dt, party_rk, became_debtor_flg, is_margin FROM
    (SELECT account_rk,  business_dt, party_rk, portf_total_amt_rub, lag(portf_total_amt_rub, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC) AS portf_total_amt_rub_yesterday,
            CASE 
                WHEN portf_total_amt_rub < 0 AND lag(portf_total_amt_rub, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC) >= 0 THEN 1
                ELSE 0
            END AS became_debtor_flg,
            margin_status_flg, 
            COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) AS margin_status_flg_yesterday,
            CASE 
                WHEN margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) = 2 THEN 1
                ELSE margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0)
            END AS is_margin
    FROM prod_v_emart.invest_account_d iad
    WHERE business_dt >= '2021-01-01' AND iad.account_status_cd = 'ACT' -- AND account_rk = '42473233'
    ORDER BY account_rk, business_dt) iad1
WHERE became_debtor_flg = 1

In [ ]:
Кол-во маржинальных БС (с вкл. флагом марж.торговли вчера и/или сегодня) каждый день
--посчитаем маржинальных клиентов (флаг марж. включён сегодня и вчера) (лучше отсортировать результаты по business_dt DESC, не перезапуская (см.время выполнения) запрос)

SELECT q.business_dt, count(account_rk) AS margin_accs_num FROM 
    (SELECT account_rk, business_dt, margin_status_flg, 
        COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) AS margin_status_flg_yesterday,
            CASE 
                WHEN margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0) = 2 THEN 1
                ELSE margin_status_flg + COALESCE(lag(margin_status_flg, 1) OVER (PARTITION BY account_rk ORDER BY business_dt ASC), 0)
            END AS is_margin
    FROM prod_v_emart.invest_account_d iad
    WHERE --iad.portf_total_amt_rub < 0 AND 
    iad.account_status_cd = 'ACT' -- AND account_rk = '42473233'
    ORDER BY account_rk, business_dt) q
WHERE q.is_margin = 1
GROUP BY q.business_dt
ORDER BY q.business_dt DESC